<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain_02_Tools_and_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 李鲁鲁学LangChain 02 Tools and Chains

这是李鲁鲁的学习笔记，原来是Sam Witteveen的笔记

增加了李鲁鲁学习时候的吐槽

因为一个独立的md文件导入知乎会更方便，所以这个课程的笔记会记录在md中

比如这节课在

https://github.com/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain%2001.md

欢迎访问[骆驼项目](https://github.com/LC1332/Luotuo-Chinese-LLM)主页和点赞

# Langchain: Tools & Chains

In [1]:
!pip -q install openai langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.2 MB/s eta 0:00:00


In [31]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-DfFyR'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Basic LLMChain - Fact Extraction

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [4]:
llm = OpenAI(model_name='text-davinci-003', 
             temperature=0, 
             max_tokens = 256)

In [5]:
article = '''Coinbase, the second-largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday, giving shareholders and market players alike an updated look into its financials. In response to the report, the company's shares are down modestly in early after-hours trading.In the fourth quarter of 2022, Coinbase generated $605 million in total revenue, down sharply from $2.49 billion in the year-ago quarter. Coinbase's top line was not enough to cover its expenses: The company lost $557 million in the three-month period on a GAAP basis (net income) worth -$2.46 per share, and an adjusted EBITDA deficit of $124 million.Wall Street expected Coinbase to report $581.2 million in revenue and earnings per share of -$2.44 with adjusted EBITDA of -$201.8 million driven by 8.4 million monthly transaction users (MTUs), according to data provided by Yahoo Finance.Before its Q4 earnings were released, Coinbase's stock had risen 86% year-to-date. Even with that rally, the value of Coinbase when measured on a per-share basis is still down significantly from its 52-week high of $206.79.That Coinbase beat revenue expectations is notable in that it came with declines in trading volume; Coinbase historically generated the bulk of its revenues from trading fees, making Q4 2022 notable. Consumer trading volumes fell from $26 billion in the third quarter of last year to $20 billion in Q4, while institutional volumes across the same timeframe fell from $133 billion to $125 billion.The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022, which resulted in Coinbase's total trading volumes and transaction revenues to fall 50% and 66% year-over-year, respectively, the company reported.As you would expect with declines in trading volume, trading revenue at Coinbase fell in Q4 compared to the third quarter of last year, dipping from $365.9 million to $322.1 million. (TechCrunch is comparing Coinbase's Q4 2022 results to Q3 2022 instead of Q4 2021, as the latter comparison would be less useful given how much the crypto market has changed in the last year; we're all aware that overall crypto activity has fallen from the final months of 2021.)There were bits of good news in the Coinbase report. While Coinbase's trading revenues were less than exuberant, the company's other revenues posted gains. What Coinbase calls its "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 of the same year, a gain of just over 34% in a single quarter.And even as the crypto industry faced a number of catastrophic events, including the Terra/LUNA and FTX collapses to name a few, there was still growth in other areas. The monthly active developers in crypto have more than doubled since 2020 to over 20,000, while major brands like Starbucks, Nike and Adidas have dived into the space alongside social media platforms like Instagram and Reddit.With big players getting into crypto, industry players are hoping this move results in greater adoption both for product use cases and trading volumes. Although there was a lot of movement from traditional retail markets and Web 2.0 businesses, trading volume for both consumer and institutional users fell quarter-over-quarter for Coinbase.Looking forward, it'll be interesting to see if these pieces pick back up and trading interest reemerges in 2023, or if platforms like Coinbase will have to keep looking elsewhere for revenue (like its subscription service) if users continue to shy away from the market.
'''

In [6]:
len(article)

3533

In [7]:
fact_extraction_prompt = PromptTemplate(
    input_variables=["text_input"],
    template="Extract the key facts out of this text. Don't include opinions. Give each fact a number and keep them short sentences. :\n\n {text_input}"
)

In [8]:
fact_extraction_chain = LLMChain(llm=llm, prompt=fact_extraction_prompt)

facts = fact_extraction_chain.run(article)

print(facts)


1. Coinbase released its Q4 2022 earnings on Tuesday.
2. Coinbase generated $605 million in total revenue in Q4 2022.
3. Coinbase lost $557 million in the three-month period on a GAAP basis.
4. Wall Street expected Coinbase to report $581.2 million in revenue.
5. Coinbase's stock had risen 86% year-to-date before its Q4 earnings were released.
6. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022.
7. Institutional volumes across the same timeframe fell from $133 billion to $125 billion.
8. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022.
9. Coinbase's total trading volumes and transaction revenues fell 50% and 66% year-over-year, respectively.
10. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022.
11. Coinbase's "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022.
12. Monthly active developers in crypto have more than double

In [ ]:
print(facts)


1. Coinbase released its Q4 2022 earnings on Tuesday.
2. Coinbase generated $605 million in total revenue in Q4 2022.
3. Coinbase lost $557 million in the three-month period on a GAAP basis.
4. Coinbase's stock had risen 86% year-to-date before its Q4 earnings were released.
5. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022.
6. Institutional volumes across the same timeframe fell from $133 billion to $125 billion.
7. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022.
8. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022.
9. Coinbase's "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022.
10. Monthly active developers in crypto have more than doubled since 2020 to over 20,000.


## Rewrite as a summary from the facts

In [9]:
investor_update_prompt = PromptTemplate(
    input_variables=["facts"],
    template="You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragrah for investors. Don't leave out key info:\n\n {facts}"
)

In [12]:
print(investor_update_prompt)

input_variables=['facts'] output_parser=None partial_variables={} template="You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragrah for investors. Don't leave out key info:\n\n {facts}" template_format='f-string' validate_template=True


In [13]:
investor_update_chain = LLMChain(llm=llm, prompt=investor_update_prompt)

investor_update = investor_update_chain.run(facts)

print(investor_update)
len(investor_update)

 the start of 2022.

Coinbase released its Q4 2022 earnings on Tuesday, reporting total revenue of $605 million, which was slightly above Wall Street's expectations of $581.2 million. Despite the positive revenue figure, Coinbase reported a GAAP loss of $557 million in the three-month period. Coinbase's stock had risen 86% year-to-date before its Q4 earnings were released. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022, while institutional volumes across the same timeframe fell from $133 billion to $125 billion. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022, leading to a 50% and 66% year-over-year decline in Coinbase's total trading volumes and transaction revenues, respectively. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022, while its "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022. Despite the overall decline 

1105

In [14]:
triples_prompt = PromptTemplate(
    input_variables=["facts"],
    template="Take the following list of facts and turn them into triples for a knowledge graph:\n\n {facts}"
)

In [15]:
triples_chain = LLMChain(llm=llm, prompt=triples_prompt)

triples = triples_chain.run(facts)

print(triples)
len(triples)

 the start of 2022.

1. (Coinbase, released, Q4 2022 earnings)
2. (Coinbase, generated, $605 million total revenue)
3. (Coinbase, lost, $557 million GAAP basis)
4. (Wall Street, expected, $581.2 million revenue)
5. (Coinbase's stock, risen, 86% year-to-date)
6. (Consumer trading volumes, fell, $26 billion Q3 2022)
7. (Institutional volumes, fell, $133 billion)
8. (Overall crypto market capitalization, fell, $1.5 trillion)
9. (Coinbase's total trading volumes, fell, 50% year-over-year)
10. (Coinbase's transaction revenues, fell, 66% year-over-year)
11. (Trading revenue, fell, $365.9 million Q3 2022)
12. (Coinbase's subscription and services revenue, rose, $210.5 million Q3 2022)
13. (Monthly active developers in crypto, more than doubled, since start of 2022)


768

## Chaining these together

In [16]:
from langchain.chains import SimpleSequentialChain, SequentialChain

full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, investor_update_chain], verbose=True)

In [17]:
response = full_chain.run(article)



> Entering new SimpleSequentialChain chain...

1. Coinbase released its Q4 2022 earnings on Tuesday.
2. Coinbase generated $605 million in total revenue in Q4 2022.
3. Coinbase lost $557 million in the three-month period on a GAAP basis.
4. Wall Street expected Coinbase to report $581.2 million in revenue.
5. Coinbase's stock had risen 86% year-to-date before its Q4 earnings were released.
6. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022.
7. Institutional volumes across the same timeframe fell from $133 billion to $125 billion.
8. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022.
9. Coinbase's total trading volumes and transaction revenues fell 50% and 66% year-over-year, respectively.
10. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022.
11. Coinbase's "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022.
12. Monthly a

In [18]:
response

' the start of 2022.\n\nCoinbase released its Q4 2022 earnings on Tuesday, reporting total revenue of $605 million, which was slightly above Wall Street\'s expectations of $581.2 million. Despite the positive revenue figure, Coinbase reported a GAAP loss of $557 million in the three-month period. Coinbase\'s stock had risen 86% year-to-date before its Q4 earnings were released. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022, while institutional volumes across the same timeframe fell from $133 billion to $125 billion. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022, leading to a 50% and 66% year-over-year decline in Coinbase\'s total trading volumes and transaction revenues, respectively. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022, while its "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022. Despite the overall de

## PAL Math Chain

In [19]:
from langchain.chains import PALChain

In [38]:
llm = OpenAI(model_name='text-davinci-003', 
             temperature=0, 
             max_tokens=512)

In [39]:
pal_chain = PALChain.from_math_prompt(llm, verbose=True)

In [40]:
print(pal_chain)

memory=None callbacks=None callback_manager=None verbose=True llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\n\n# solution in Python:\n\n\ndef solution():\n    """Olivia has $23. She bought five bagels for $3 each. How much money does she have left?"""\n    money_initial = 23\n    bagels = 5\n    bagel_cost = 3\n    money_spent = bagels * bagel_cost\n    money_left = money_initial - money_spent\n    result = money_left\n    return result\n\n\n\n\n\nQ: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\n\n# solution in Python:\n\n\ndef solution():\n    """Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf ba

In [42]:
question_02= "The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more, how many apples do they have?"

In [43]:
pal_chain.run(question_02)



> Entering new PALChain chain...
def solution():
    """The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more, how many apples do they have?"""
    apples_initial = 23
    apples_used = 20
    apples_bought = 6
    apples_total = apples_initial + apples_bought - apples_used
    result = apples_total
    return result

> Finished chain.


'9'

In [44]:
question = "Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"

In [45]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"""
    cindy_pets = 4
    marcia_pets = cindy_pets + 2
    jan_pets = marcia_pets * 3
    total_pets = cindy_pets + marcia_pets + jan_pets
    result = total_pets
    return result

> Finished chain.


'28'

## API Chains - OpenMeteo - Weather information

can throw errors based on API return length

In [46]:
from langchain import OpenAI
from langchain.chains.api.prompt import API_RESPONSE_PROMPT

from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate


In [47]:
llm = OpenAI(temperature=0,
             max_tokens=100)

In [51]:
print(open_meteo_docs.OPEN_METEO_DOCS)

BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit	String	No	celsius	If fahrenheit is set, al

In [48]:
from langchain.chains.api import open_meteo_docs
chain_new = APIChain.from_llm_and_api_docs(llm, 
                                           open_meteo_docs.OPEN_METEO_DOCS, 
                                           verbose=True)

In [49]:
chain_new.run('What is the temperature like right now in Bedok, Singapore in degrees Celcius?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=1.3&longitude=103.9&hourly=temperature_2m&current_weather=true&temperature_unit=celsius
{"latitude":1.375,"longitude":103.875,"generationtime_ms":0.14197826385498047,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":6.0,"current_weather":{"temperature":28.1,"windspeed":9.2,"winddirection":225.0,"weathercode":80,"is_day":1,"time":"2023-05-29T01:00"},"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2023-05-29T00:00","2023-05-29T01:00","2023-05-29T02:00","2023-05-29T03:00","2023-05-29T04:00","2023-05-29T05:00","2023-05-29T06:00","2023-05-29T07:00","2023-05-29T08:00","2023-05-29T09:00","2023-05-29T10:00","2023-05-29T11:00","2023-05-29T12:00","2023-05-29T13:00","2023-05-29T14:00","2023-05-29T15:00","2023-05-29T16:00","2023-05-29T17:00","2023-05-29T18:00","2023-05-29T19:00","2023-05-29T20:00","2023-05-29T21:00","2023-05-29T22:00","2023-05-29T23:00","202

' The temperature right now in Bedok, Singapore is 28.1°C.'

In [52]:
chain_new.run('Is it raining in Singapore?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=1.3521&longitude=103.8198&current_weather=true&rain=true
{"latitude":1.375,"longitude":103.875,"generationtime_ms":0.11491775512695312,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":46.0,"current_weather":{"temperature":27.8,"windspeed":9.2,"winddirection":225.0,"weathercode":80,"is_day":1,"time":"2023-05-29T01:00"}}

> Finished chain.


' It is currently raining in Singapore, with a temperature of 27.8°C and a windspeed of 9.2 km/h.'

In [ ]:
# chain_new.